In [44]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Embedding, Flatten, Input, concatenate
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import keras.backend as K
import numpy as np
import pandas as pd
import pickle
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

def zuhe(line):
    line['hour'] = line['hour'].astype(str)
    line['weekday'] = line['weekday'].astype(str)
    line['is_peek'] = line['is_peek'].astype(str)
    line['is_workday'] = line['is_workday'].astype(str)
                                   
    line['hour_weekday'] = line['hour'] + line['weekday']
    line['peak_workday'] = line['is_peek'] + line['is_workday']
    
    line['hour'] = line['hour'].astype(int)
    line['is_peek'] = line['is_peek'].astype(int)
    line['is_workday'] = line['is_workday'].astype(int)
    line['weekday'] = line['weekday'].astype(int)
    
    line['hour_weekday'] = line['hour_weekday'].astype(int)
    line['peak_workday'] = line['peak_workday'].astype(int)
    return line

with open('./data/train-id4-crowd-grid4.txt', 'rb') as data_file:
    train = pickle.load(data_file)
    #训练验证都划分
    #900 29 26 #800 27.9 25.9 #700 27.5 25.3 #27.4 24.8 #26.4 24.08 #23 21 #(90)250 21 19.5 #(80)17. 16.5
    #训练划分 验证不划分
    #250
train = train[(train['Diff_Time']<600)]
train = zuhe(train)
with open('./data/test-id4-crowd-grid4.txt', 'rb') as data_file:
    test = pickle.load(data_file)
test = zuhe(test)
from sklearn import preprocessing
x1 = train['ID'].as_matrix().reshape([-1,1])
N1 = x1.shape[0]

x2 = test['ID'].as_matrix().reshape([-1,1])
N2 = x2.shape[0]
x = np.concatenate((x1,x2))
x = preprocessing.LabelEncoder().fit_transform(x) #13963746x21567
print(x.shape)#(14454126, 21567)
train['new_ID'] = x[:N1]
test['new_ID'] = x[N1:]

/home/kawayi-4/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:43: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/kawayi-4/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:46: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/kawayi-4/anaconda3/envs/python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(14223933,)


In [45]:
col1 = ['new_ID']
col2 = [c for c in train if
       c not in ['Unnamed: 0','ID2', 's_x', 's_y', 'e_x', 'e_y', 'new_ID','ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
                 'Diff_Time']]
col3 = ['Source_Station_encode']
col4 = ['Target_Station_encode']

X_train1 = train[col1].values
X_train2 = train[col2].values
X_train3 = train[col3].values
X_train4 = train[col4].values
y_train = train['Diff_Time'].values

print(col1,col2,col3,col4)
print('train',X_train1.shape, X_train2.shape, X_train3.shape, X_train4.shape)

col1 = ['new_ID']
col2 = [c for c in test if
       c not in ['Unnamed: 0','ID2', 's_x', 's_y', 'e_x', 'e_y', 'new_ID','ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
                 'Diff_Time','Distance1', 'distance2','TERMINALNO', 'new_dist']]
col3 = ['Source_Station_encode']
col4 = ['Target_Station_encode']

print(col1,col2,col3,col4)
X_test1 = test[col1].values
X_test2 = test[col2].values
X_test3 = test[col3].values
X_test4 = test[col4].values
print('test',X_test1.shape,X_test2.shape,X_test3.shape,X_test4.shape)

['new_ID'] ['Distance', 'hour', 'is_peek', 'weekday', 'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'h_aver_diff', 'h_aver_d', 'h_aver_v', 'is_crowd', 's_ij', 'e_ij', 'grid_aver_diff', 'grid_aver_d', 'Source_Station_encode', 'Target_Station_encode', 'hour_weekday', 'peak_workday'] ['Source_Station_encode'] ['Target_Station_encode']
train (13733553, 1) (13733553, 22) (13733553, 1) (13733553, 1)
['new_ID'] ['Distance', 'hour', 'is_peek', 'weekday', 'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'h_aver_diff', 'h_aver_d', 'h_aver_v', 'is_crowd', 's_ij', 'e_ij', 'grid_aver_diff', 'grid_aver_d', 'Source_Station_encode', 'Target_Station_encode', 'hour_weekday', 'peak_workday'] ['Source_Station_encode'] ['Target_Station_encode']
test (490380, 1) (490380, 22) (490380, 1) (490380, 1)


In [3]:
from keras.layers import Reshape, Conv2D, MaxPooling2D
def get_model():
    input1 = Input(shape=(1,))
    input2 = Input(shape=(20,))
    input3 = Input(shape=(1,))
    input4 = Input(shape=(1,))

    x1 = Embedding(21064, 128, input_length=1)(input1)
    x1 = Flatten()(x1)
    x2 = Dense(units=128)(input2)
    x2 = Activation("relu")(x2)
    x3 = Embedding(4609, 72, input_length=1)(input3)
    x3 = Flatten()(x3)
    x4 = Embedding(4609, 72, input_length=1)(input4)
    x4 = Flatten()(x4)
    
    x = concatenate([x1, x2, x3, x4])
    #CNN
    x = Reshape((16, 16, 1))(x)
    x = Conv2D(32, (3, 3), padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    # x = Dense(512, activation='relu')(x)
    # x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(1, activation=None)(x)
    model = Model(inputs=[input1, input2, input3, input4], outputs=out)
    print(model.summary())
    # model.load_weights('./log/weights-21.7671.hdf5')
    model.compile(loss=root_mean_squared_error, optimizer='adam', metrics=[root_mean_squared_error])
    return model

from sklearn.model_selection import StratifiedKFold, KFold

def get_stacking(X_train1, X_train2, labels, n_folds=10):
    train_num, test_num = X_train1.shape[0], X_test1.shape[0]
    second_level_train_set1 = np.zeros((train_num,))
    second_level_test_set1 = np.zeros((test_num,))
    test_nfolds_sets1 = np.zeros((test_num, n_folds))
    
    second_level_train_set2 = np.zeros((train_num,))
    second_level_test_set2 = np.zeros((test_num,))
    test_nfolds_sets2 = np.zeros((test_num, n_folds))

    folds = KFold(n_splits=n_folds, shuffle=True, random_state=1001001)
        
    for n_fold,(train_idx, val_idx) in enumerate(folds.split(labels)):
        train_1 = X_train1[train_idx]
        train_2 = X_train2[train_idx]
        train_y = labels[train_idx]

        val_1 = X_train1[val_idx]
        val_2 = X_train2[val_idx]
        val_y = labels[val_idx]
#         ckpt_path = './logtianchi/'+ model_list[n_fold]
#         print(ckpt_path)
        model = get_model()
        if n_fold == 0:
            print(model.summary())
        print(n_fold)
        model.fit([train_1,train_2], train_y, validation_data=([val_1,val_2], val_y), epochs=10, batch_size=128,callbacks=[ckpt,earlyStopping])
        
        second_level_train_set[val_idx] = model.predict([val_1, val_2], batch_size=1024)[:, 0]
        test_nfolds_sets[:,n_fold] = model.predict([test_q1_data, test_q2_data, test_magic], batch_size=1024)[:, 0]
    
    "---save---"
    with open('./data/test_nfolds_sets.txt', 'wb') as data_file:
        pickle.dump(test_nfolds_sets, data_file)

    second_level_test_set1[:] = test_nfolds_sets1.mean(axis=1)

    result = second_level_test_set
    
    return second_level_train_set, second_level_test_set, result

In [4]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)+1e-15, axis=-1))
earlyStopping=EarlyStopping(monitor='val_loss', patience=4, verbose=0, mode='auto')

ckpt_path = './log/weights-{val_loss:.4f}.hdf5'
ckpt = ModelCheckpoint(ckpt_path,
                       monitor='val_loss', 
                       verbose=1, 
                       save_best_only=True, 
                       mode='min')

In [5]:
# satck
train_sets = []
test_sets = []
train_set, test_set, result = get_stacking(X_train1, X_train2, y_train, n_folds=10)
train_sets.append(train_set)
test_sets.append(test_set)


meta_train = np.concatenate([result_set.reshape(-1,1) for result_set in train_sets], axis=1)
meta_test = np.concatenate([y_test_set.reshape(-1,1) for y_test_set in test_sets], axis=1)
print(meta_train.shape, meta_test.shape)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 128)       2696192     input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          2688        input_2[0][0]                    
__________________________________________________________________________________________________
flatten_1 

KeyboardInterrupt: 

In [7]:
"---save---"
with open('./data/meta_train_15char1.txt', 'wb') as data_file:
    pickle.dump(meta_train, data_file)
with open('./data/meta_test_15char1.txt', 'wb') as data_file:
    pickle.dump(meta_test, data_file)

In [46]:
import numpy as np
import pandas as pd
import pickle
"---load---"
with open('./data/meta_train_nn2.txt', 'rb') as data_file:
    meta_train = pickle.load(data_file)
with open('./data/meta_test_nn2.txt', 'rb') as data_file:
    meta_test = pickle.load(data_file)
print(meta_train.shape,meta_test.shape)

(13733553, 1) (490380, 1)


In [33]:
with open('./data/test_nfolds_sets.txt', 'rb') as data_file:
    res_test = pickle.load(data_file)
cv10res = res_test.mean(axis=1)
cv10res.shape

(490380,)

In [48]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
X_train,X_dev, y_train, y_dev = train_test_split(meta_train,y_train,test_size=0.1, random_state=42)
print(X_train.shape,X_dev.shape)
lgb_train  = lgb.Dataset(X_train, y_train,)#feature_name=col,categorical_feature=['ID']
lgb_eval = lgb.Dataset(X_dev, y_dev,reference=lgb_train)
params = {'num_leaves':40, 
          'max_depth':8,
          'seed':2018,
          'colsample_bytree':0.8,
          'subsample':0.9,
          'num_threads':25,
          'n_estimators':1000,
          'learning_rate': 0.1,
          'objective':'regression_l2',
          'metric':'rmse'}
gbm = lgb.train(params, lgb_train,early_stopping_rounds=200,valid_sets=lgb_eval)

(12360197, 1) (1373356, 1)


/home/kawayi-4/.local/lib/python3.6/site-packages/lightgbm/engine.py:105: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's rmse: 78.8835
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's rmse: 72.99
[3]	valid_0's rmse: 67.8403
[4]	valid_0's rmse: 63.3628
[5]	valid_0's rmse: 59.4891
[6]	valid_0's rmse: 56.1553
[7]	valid_0's rmse: 53.3017
[8]	valid_0's rmse: 50.8732
[9]	valid_0's rmse: 48.8172
[10]	valid_0's rmse: 47.0859
[11]	valid_0's rmse: 45.6354
[12]	valid_0's rmse: 44.4257
[13]	valid_0's rmse: 43.4211
[14]	valid_0's rmse: 42.5901
[15]	valid_0's rmse: 41.9049
[16]	valid_0's rmse: 41.3416
[17]	valid_0's rmse: 40.8796
[18]	valid_0's rmse: 40.5016
[19]	valid_0's rmse: 40.1928
[20]	valid_0's rmse: 39.9411
[21]	valid_0's rmse: 39.7361
[22]	valid_0's rmse: 39.5693
[23]	valid_0's rmse: 39.4336
[24]	valid_0's rmse: 39.3235
[25]	valid_0's rmse: 39.2341
[26]	valid_0's rmse: 39.1616
[27]	valid_0's rmse: 39.1028
[28]	valid_0's rmse: 39.0552
[29]	valid_0's rmse: 39.0166
[30]	valid_0's rmse: 38.9853
[31]	valid_0's rmse: 38.96
[32]	valid_0's rmse: 38.9395
[33]	valid_0's rmse: 3

In [21]:
final_result = gbm.predict(meta_test, num_iteration=gbm.best_iteration)

In [34]:
final_result = cv10res

In [35]:
with open('./data/test-id4-crowd-grid4.txt', 'rb') as data_file:
    test = pickle.load(data_file)
test = zuhe(test)
test = test[['new_dist', 'ID', 'O_LINENO', 'O_UP', 'Source_Station', 'Target_Station',
       'Distance', 'distance2', 'O_TIME', 'hour', 'is_peek', 'weekday',
       'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'max_v',
       'h_aver_diff', 'h_aver_d', 'h_aver_v', 'TERMINALNO', 
       'hour_weekday']]
test.columns=['Distance', 'ID', 'O_LINENO', 'O_UP', 'Source_Station', 'Target_Station',
       'Distance1', 'distance2', 'O_TIME', 'hour', 'is_peek', 'weekday',
       'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'max_v',
       'h_aver_diff', 'h_aver_d', 'h_aver_v', 'TERMINALNO', 
       'hour_weekday']

test['pred1'] = final_result
test['pred2'] = final_result

sub1 = test[['O_LINENO','TERMINALNO', 'O_UP','Source_Station','Target_Station','O_TIME','pred1','pred2','Distance','Distance1']]
sub1.columns = ['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime','pred1','pred2','Distance','Distance1']
sub1 = sub1.reset_index()
del sub1['index']

In [36]:
sub=pd.read_csv("./toBePredicted_0607_segment.csv", sep=",")
# sub['realTime'] = pd.to_datetime(sub['realTime'],format='%Y-%m-%d %H:%M:%S')
sub2 = sub[['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime','distance']]
sub2=pd.merge(sub2,sub1,on=['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime'],how='left')
sub2

,LINE,TERMINALNO,UP,pred_start_stop_ID,pred_end_stop_ID,realTime,distance,pred1,pred2,Distance,Distance1
0,678,904388,0,11,12,2017-10-28 14:00:00,0.417984,125.154091,125.154091,0.679068,1.097053
1,678,904388,0,12,13,2017-10-28 14:00:00,0.000000,153.527003,153.527003,0.744740,0.744740
2,678,904388,0,13,14,2017-10-28 14:00:00,0.000000,68.068784,68.068784,0.479191,0.479191
3,678,904388,0,14,15,2017-10-28 14:00:00,0.000000,89.885497,89.885497,0.367919,0.367919
4,678,904388,0,15,16,2017-10-28 14:00:00,0.000000,237.310681,237.310681,1.141722,1.141722
5,678,904388,0,16,17,2017-10-28 14:00:00,0.000000,142.010307,142.010307,0.578620,0.578620
6,678,904388,0,17,18,2017-10-28 14:00:00,0.000000,75.361789,75.361789,0.410412,0.410412
7,678,904388,0,18,19,2017-10-28 14:00:00,0.000000,118.533014,118.533014,0.665032,0.665032
8,678,904388,0,19,20,2017-10-28 14:00:00,0.000000,183.262018,183.262018,0.907266,0.907266
9,678,904388,0,20,21,2017-10-28 14:00:00,0.000000,141.353639,141.353639,0.471062,0.471062


In [37]:
sub2['div_dist'] = sub2['Distance'] / sub2['Distance1']
sub2['new_pred'] = sub2['div_dist'] * sub2['pred1']

import math
for i in range(sub2.shape[0]):
    s = sub2.iloc[i]
    if math.isnan(s['div_dist']):
        sub2.loc[i,'new_pred'] = s['pred1']

#计算用newpred和直接用新距离之间的偏差
print((sub2['pred1']-sub2['pred2']).mean())
print((sub2['pred2']-sub2['new_pred']).mean())
print((sub2['pred1']-sub2['new_pred']).mean())

0.0
4.328145208428728
4.328145208428728


In [38]:
sub2['new_pred'].mean()

126.75228316967771

In [39]:
sub2.to_csv('./toBePredicted_0808_resultn_cv10.csv',sep=",",index=False)#0605 29.2467

print('finished')

finished
